# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 79 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.20444


extracting tarball to tmp_2509.20444... done.
Retrieving document from  https://arxiv.org/e-print/2509.20621


extracting tarball to tmp_2509.20621... done.


Found 69 bibliographic references in tmp_2509.20621/main.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2509.21299


extracting tarball to tmp_2509.21299... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.20621-b31b1b.svg)](https://arxiv.org/abs/2509.20621) | **Constraints on the Orbit of the Young Substellar Companion GQ Lup B from High-Resolution Spectroscopy and VLTI/GRAVITY Astrometry**  |
|| V. Venkatesan, et al. -- incl., <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-09-26*|
|*Comments*| *15 pages, 5 figures, Accepted to ApJ*|
|**Abstract**|            Understanding the orbits of giant planets is critical for testing planet formation models, particularly at wide separations greater than 10 au where traditional core accretion becomes inefficient. However, constraining orbits at these separations has been challenging because of sparse orbital coverage and degeneracies in the orbital parameters. We use existing high-resolution spectroscopic measurements from CRIRES+ (R ~ 100000), astrometric data from SPHERE, NACO, and ALMA, and new high-precision GRAVITY astrometry to refine the orbit of GQ Lup B, a ~30 M_J companion at ~100 au, in a system that also hosts a circumstellar disk and a wide companion, GQ Lup C. Including radial velocity data significantly improves orbital constraints by breaking the degeneracy between inclination and eccentricity that affects astrometry-only fits for long-period companions. This work is among the first to combine high-precision astrometry with the companion's relative radial velocity to achieve improved orbital constraints. The eccentricity is refined from e = 0.47 (+0.14, -0.16) with GRAVITY alone to e = 0.35 (+0.10, -0.09) when RVs and GRAVITY data are combined. The orbit is misaligned by 63 (+6, -14) deg relative to the circumstellar disk and 52 (+19, -24) deg relative to the host star spin axis, and is more consistent (34 (+6, -13) deg) with the inclination of the wide tertiary companion GQ Lup C disk. These results support a formation scenario for GQ Lup B consistent with cloud fragmentation and highlight the power of combining companion RV constraints with interferometric astrometry to probe the dynamics and formation of wide-orbit substellar companions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.20444-b31b1b.svg)](https://arxiv.org/abs/2509.20444) | **SAGAbg III: Environmental Stellar Mass Functions, Self-Quenching, and the Stellar-to-Halo Mass Relation in the Dwarf Galaxy Regime**  |
|| E. Kado-Fong, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-26*|
|*Comments*| *34 pages, 12 figures, 6 pages; submitted to ApJ*|
|**Abstract**|            Recent efforts have extended our view of the number and properties of satellite galaxies beyond the Local Group firmly down to $\rm M_\star\sim 10^6 M_\odot$. A similarly complete view of the field dwarf population has lagged behind. Using the background galaxies sample from the Satellites Around Galactic Analogs (SAGA) Survey at $z<0.05$, we take inventory of the dwarf population down to $\rm M_\star \sim 5\times10^6 M_\odot$ using three metrics: the stellar mass function (SMF) as function of environment, the stellar-to-halo mass relation (SHMR) of dwarf galaxies inferred via abundance matching, and the quenched fraction of highly isolated dwarfs. We find that the low-mass SMF shape shows minimal environmental dependence, with the field dwarf SMF described by a low-mass power-law index of $\alpha_1=-1.44\pm0.09$ down to $\rm M_\star \sim 5\times10^6 M_\odot$, and that the quenched fraction of isolated dwarfs drops monotonically to $f_{q} \sim 10^{-3}$ at $\rm M_\star \sim \rm 10^{8.5} M_\odot$. Though slightly steeper than estimates from \HI{} kinematic measures, our inferred SHMR agrees with literature measurements of satellite systems, consistent with minimal environmental dependence of the SHMR in the probed mass range. Finally, although most contemporary cosmological simulations against which we compare accurately predict the \sagalocal{} SHMR, we find that big-box cosmological simulations largely over-predict isolated galaxy quenched fractions via a turnaround in $f_q(\rm M_\star)$ at $\rm 10^8\lesssim M_\star/M_\odot\lesssim 10^9$, underscoring the complexities in disentangling the drivers of galaxy formation and the need for systematic multidimensional observations of the dwarf population across environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.21299-b31b1b.svg)](https://arxiv.org/abs/2509.21299) | **Outflow-cloud interaction as the possible origin of the peculiar radio emission in the tidal disruption event AT2018cqh**  |
|| L. Yang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-26*|
|*Comments*| *12 pages, 6 figures, accepted for publication in ApJL*|
|**Abstract**|            AT2018cqh is a unique optical tidal disruption event (TDE) discovered in a dwarf galaxy exhibiting delayed X-ray and radio flares. We present the results from high-resolution VLBA and e-MERLIN radio observations of AT2018cqh extending to $\delta$t $\sim$ 2250 days post discovery, which reveal a compact radio emission, unresolved at a scale of <~ 0.13 pc at 7.6 GHz, with a high brightness temperature of $T_b$ ~> 4.03 $\times$ 10$^{9}$ K. The radio spectral energy distribution (SED) is found to gradually shift towards a higher peak flux density and frequency over a period of $\sim$1000 days. An equipartition analysis suggests that there is a little change in the radio emitting region over this period, while the electron density increases by a factor of 3. The radio light curve at 0.89 GHz continues to rise, with a bump feature lasting for 240 days. These properties are in contrast to the predictions of standard shockwave model from a diffuse circumnuclear medium, but could be explained if dense clouds exist in the circumnuclear environment. The latter scenario is supported by our hydrodynamic simulations of the interaction of TDE outflow with a cloud, which can reproduce the temporal evolution in the radio SED. This work highlights the importance of the outflow-cloud interaction in explaining the delayed, fast-rising radio emission observed in some TDEs, especially those occurring in galaxies with pre-existing AGN activity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.20621/./ecc_inc_4panel.png', 'tmp_2509.20621/./combined_contour.png', 'tmp_2509.20621/./updated_orbit.png']
copying  tmp_2509.20621/./ecc_inc_4panel.png to _build/html/
copying  tmp_2509.20621/./combined_contour.png to _build/html/
copying  tmp_2509.20621/./updated_orbit.png to _build/html/
exported in  _build/html/2509.20621.md
    + _build/html/tmp_2509.20621/./ecc_inc_4panel.png
    + _build/html/tmp_2509.20621/./combined_contour.png
    + _build/html/tmp_2509.20621/./updated_orbit.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Constraints on the Orbit of the Young Substellar Companion GQ Lup B from High-Resolution Spectroscopy and VLTI/GRAVITY Astrometry $\footnote{Based on observations collected at the European Southern Observatory under ESO programmes 1104.C-0651 and 109.238N.001.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.20621-b31b1b.svg)](https://arxiv.org/abs/2509.20621)<mark>Appeared on: 2025-09-26</mark> -  _15 pages, 5 figures, Accepted to ApJ_

</div>
<div id="authors">

V. Venkatesan, et al. -- incl., <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Understanding the orbits of giant planets is critical for testing planet formation models, particularly at wide separations ( $>$ 10 au) where traditional core accretion becomes inefficient. However, constraining orbits at these separations has historically been challenging due to sparse orbital coverage and related degeneracies in the orbital parameters. In this work, we use existing high-resolution ( $R \sim 100{,}000$ ) spectroscopic measurements from CRIRES+, astrometric data from SPHERE, NACO, and ALMA, and combine it with new high-precision GRAVITY astrometry data to refine the orbit of GQ Lup B, a $\sim$ 30 $M_{\mathrm{J}}$ companion at $\sim$ 100 au, in a system that also hosts a circumstellar disk and a wide companion, GQ Lup C. Including RV data significantly improves orbital constraints by breaking the degeneracy between inclination and eccentricity that plagues astrometry-only fits for long-period companions. Our work is one of the first to combine high-precision astrometry with the companion’s relative radial velocity measurements to achieve significantly improved orbital constraint. The eccentricity is refined from $e = 0.47^{+0.14}_{-0.16}$ (GRAVITY only) to $e = 0.35^{+0.10}_{-0.09}$ when RVs and GRAVITY data are combined. We also compute the mutual inclinations between the orbit of GQ Lup B, the circumstellar disk, the stellar spin axis, and the disk of GQ Lup C. The orbit is misaligned by $63^{+6}_{-14}^\circ$ relative to the circumstellar disk, $52^{+19}_{-24}^\circ$ with the host star's spin axis, but appears more consistent ( $34^{+6}_{-13}^\circ$ ) with the inclination of the wide tertiary companion GQ Lup C's disk. These results support a formation scenario for GQ Lup B consistent with cloud fragmentation. They highlight the power of combining companion RV constraints with interferometric astrometry to probe the dynamics and formation of wide-orbit substellar companions.

</div>

<div id="div_fig1">

<img src="tmp_2509.20621/./ecc_inc_4panel.png" alt="Fig4" width="100%"/>

**Figure 4. -** Posterior eccentricity and inclination distributions for GQ Lup B based on six orbital fits (see Table \ref{tab:fit_cases}), grouped by inclusion of GRAVITY astrometry. The histograms show the relative probability of eccentricity and inclination values. Inclusion of GRAVITY and relative RV measurements substantially tightens the constraints. While the high-precision RV case (orange) provides the narrowest distribution, it may be biased due to stellar variability or unresolved companions. The adopted blue case ({\texttt{GRAVLowRV}}) incorporates GRAVITY astrometry and low-precision RV, offering a robust and physically plausible solution consistent with a moderately eccentric and inclined orbit. (*fig:ecc*)

</div>
<div id="div_fig2">

<img src="tmp_2509.20621/./combined_contour.png" alt="Fig1" width="100%"/>

**Figure 1. -** 1$\sigma$ and 2$\sigma$ joint posterior distributions of eccentricity and inclination for GQ Lup B under four cases (see Table \ref{tab:fit_cases}) are shown by lines and dashed lines, respectively. The inclusion of GRAVITY astrometry significantly narrows the orbital parameter space, particularly by reducing the volume of the joint eccentricity-inclination posterior. Adding radial velocity measurements—especially in the {{\texttt{GRAVLowRV}}} and {{\texttt{GRAVHighRV}}} cases—further sharpens the constraints by reducing the eccentricity-inclination degeneracy, yielding a well-localized solution consistent with a moderately eccentric orbit. (*fig:contours*)

</div>
<div id="div_fig3">

<img src="tmp_2509.20621/./updated_orbit.png" alt="Fig2" width="100%"/>

**Figure 2. -** Orbit fit for Case \texttt{GRAVLowRV}(see Table \ref{tab:fit_cases}), using astrometry including GRAVITY data and low-precision radial velocity measurement. The plot shows posterior samples from the orbit fit for GQ Lup B, generated using \texttt{orbitize!}. Blue ellipses represent 100 orbits randomly drawn from the posterior, and red points indicate the relative astrometric data. In the right panels, red points correspond to astrometric measurements from both the literature and new GRAVITY data included in the fit; gray lines trace the same 100 posterior orbits in separation and position angle. The results illustrate the power of GRAVITY data in improving the orbit solution for GQ Lup B. (*fig:orbit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.20621"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

122  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
